In [3]:
import fitz

In [13]:
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()

client = OpenAI( api_key=os.environ.get("OPENAI_API_KEY"))

In [14]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2,api_key=os.environ.get("TAVILY_API_KEY"),include_domains = ['www.healthyschoolrecipes.com'])
search_results = search.invoke("what are the ingredients recipe for Stromboli")

In [15]:
def make_system_prompt() -> str:
    return (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Use the provided tools to progress towards answering the question."
        " Do NOT answer based on your own knowledge or assumptions."
        " ONLY respond if the tool provides a valid and relevant result."
        " If the tool fails or returns nothing useful, say you cannot find the answer."
        " If you or any of the other assistants have the final answer or deliverable,"
        " prefix your response with FINAL ANSWER so the team knows to stop."
    )

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o", model_provider="openai",api_key=os.environ.get("OPENAI_API_KEY"))
search = TavilySearchResults(max_results=2,include_domains = ['www.healthyschoolrecipes.com'])
tools = [search]

agent_executor = create_react_agent(model, tools,
                                    prompt=make_system_prompt)

NameError: name 'make_system_prompt' is not defined

In [17]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

# 1. Setup Tavily tool
search = TavilySearchResults(
    max_results=2,
    include_domains=["www.healthyschoolrecipes.com"]  # Optional
)

# 2. Define system prompt
def make_system_prompt():
    return (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Use the provided tools to progress towards answering the question."
        " Do NOT answer based on your own knowledge or assumptions."
        " ONLY respond if the tool provides a valid and relevant result."
        " If the tool fails or returns nothing useful, say you cannot find the answer."
        " If you or any of the other assistants have the final answer or deliverable,"
        " prefix your response with FINAL ANSWER so the team knows to stop."
         )

# 3. Setup LLM and agent
# llm = ChatOpenAI(model="gpt-4o", temperature=0)
model = init_chat_model("gpt-4o", model_provider="openai",api_key=os.environ.get("OPENAI_API_KEY"))
# Create agent with tool
agent_executor = create_react_agent(
    model,
    tools=[search],
    prompt=make_system_prompt()
)

# 4. Run query
query = "Fetch all the detailed ingredients for any idly recipe?"
response = agent_executor.invoke(
    {"messages": [HumanMessage(content=query)]}
)

print(response["messages"][-1].content)

I cannot find the detailed ingredients for an idly recipe.


In [ ]:
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.graph import MessagesState


from typing import Literal
GRADE_PROMPT = (
    "You are a grader assessing relevance of a retrieved document to a user question. \n "
    "Here is the retrieved document: \n\n {context} \n\n"
    "Here is the user question: {question} \n"
    "If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n"
    "Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."
    )

class RouteQuery(BaseModel):
    """Binary score to assess answer addresses question."""
    binary_score: Literal["yes", "no"] = Field(
        ...,
        description="Given a user question choose to route it to yes or a no.",
    )

def toolagent(state: MessagesState):

    query = state["messages"][0].content
    response = agent_executor.invoke({"messages": [HumanMessage(content=query)]})

    context = response["messages"][-1].content
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    # structured_llm_grader = llm.with_structured_output(RouteQuery)

    # system = """You are a grader assessing whether the answer addresses / resolves a question \n 
    #     Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question."""
    # answer_prompt = ChatPromptTemplate.from_messages(
    #     [
    #         ("system", system),
    #         ("human", "User question: \n\n {question} \n\n LLM answer: {answer}"),
    #     ]
    # )

    # answer_grader = answer_prompt | structured_llm_grader
    # answer_grader.invoke({"question": query, "answer": context })
    prompt = GRADE_PROMPT.format(question=query, context=context)
    response = (
        llm
        # highlight-next-line
        .with_structured_output(RouteQuery).invoke(
            [{"role": "user", "content": prompt}]
        )
    )
    score = response.binary_score

    if score == "yes":
        return {"question": query, "generation": context}
    else:
        return {"question": query}

change of plan

In [17]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

# 1. Setup Tavily tool

def toolagent(state):
    search = TavilySearchResults(
        max_results=2,
        include_domains=['www.indianhealthyrecipes.com','www.allrecipes.com']
    )

    # 2. Define system prompt
    def make_system_prompt():
        return (
            "You are a helpful AI assistant, collaborating with other assistants."
            " Use the provided tools to progress towards answering the question."
            " Do NOT answer based on your own knowledge or assumptions."
            " ONLY respond if the tool provides a valid and relevant result."
            " If the tool fails or returns nothing useful, say you cannot find the answer."
            " Also site the source where the recipe had been taken from."
            )

    # 3. Setup LLM and agent
    # llm = ChatOpenAI(model="gpt-4o", temperature=0)
    model = init_chat_model("gpt-4o", model_provider="openai",api_key=os.environ.get("OPENAI_API_KEY"))
    # Create agent with tool
    agent_executor = create_react_agent(
        model,
        tools=[search],
        prompt=make_system_prompt()
    )

    # 4. Run query
    query = state['question']
    response = agent_executor.invoke(
        {"messages": [HumanMessage(content=query)]}
    )

    # print(response["messages"][-1].content)
    return {"generation":response["messages"][-1].content}

In [18]:
def regular_agent(state):
    query= state['question']
    client = OpenAI( api_key=os.environ.get("OPENAI_API_KEY"))
    completion = client.chat.completions.create(
    # model="gpt-4o",
    model="gpt-4o-mini",
    # model="gpt-4.1",
    # model="gpt-4.1-mini",
    # model="gpt-4.1-nano",
    # model="o3-mini",
    messages=[
            {"role": "system",
            "content":"You are a cooking agent, Help me generate the ingredients for the user provided recipe"           
            },
            {
                  "role": "user",
                  "content": f"""{query}"""
                  
            },
      ],
      )
    return {
        "generation": completion.choices[0].message.content
        }


In [19]:
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.graph import MessagesState


from typing import Literal
GRADE_PROMPT = (
    "You are a grader assessing relevance of a retrieved document to a user question. \n "
    "Here is the retrieved document: \n\n {context} \n\n"
    "Here is the user question: {question} \n"
    "If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n"
    "Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."
    )

class RouteQuery(BaseModel):
    """Binary score to assess answer addresses question."""
    binary_score: Literal["yes", "no"] = Field(
        ...,
        description="Given a user question choose to route it to yes or a no.",
    )

def answersatisfied(state):

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    query = state['question']
    generation = state['generation']
    prompt = GRADE_PROMPT.format(question=query, context=generation)
    response = (
        llm
        # highlight-next-line
        .with_structured_output(RouteQuery).invoke(
            [{"role": "user", "content": prompt}]
        )
    )
    score = response.binary_score

    if score == "yes":
        return "useful"
    else:
        return "not useful"

In [20]:
from typing import List
from typing_extensions import TypedDict
from langgraph.graph import END, StateGraph, START


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
    """
    question: str
    generation: str

workflow = StateGraph(GraphState)
workflow.add_node(toolagent)
workflow.add_node(regular_agent)

workflow.add_edge(START, "toolagent")
workflow.add_conditional_edges(
    "toolagent",
    answersatisfied,
    {
        "useful": END,
        "not useful":"regular_agent"
    }
)
workflow.add_edge("regular_agent",END)
app = workflow.compile()




In [ ]:
inputs = {"question": "Fetch all the detailed ingredients for Baked Shrimp Scampi?"}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        print(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    print("\n---\n")

# Final generation
print(value["generation"])


Node 'toolagent':

---

Here are the detailed ingredients for two different Baked Shrimp Scampi recipes:

1. **[Shrimp Scampi Bake Recipe from Allrecipes](https://www.allrecipes.com/recipe/25874/shrimp-scampi-bake/)**

   Ingredients:
   - 2 pounds medium raw shrimp, shelled, deveined, with tails attached
   - 1 cup butter
   - 2 tablespoons Dijon mustard
   - 1 tablespoon fresh lemon juice
   - 1 tablespoon chopped garlic
   - 1 tablespoon chopped fresh parsley

2. **[Baked Shrimp Scampi Recipe from Allrecipes](https://www.allrecipes.com/recipe/229959/baked-shrimp-scampi/)**

   Ingredients:
   - 1 pound large shrimp, peeled and deveined
   - 1 cup unsalted butter
   - ¼ cup white wine
   - 2 tablespoons lemon juice
   - 2 tablespoons dried parsley
   - 1 teaspoon cayenne pepper
   - 2 tablespoons minced garlic
   - ½ cup Italian-seasoned bread crumbs

You may choose either recipe based on your preference for ingredients.


: 